In [1]:
from weight_avg_trees_best import *
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from tqdm import tqdm
device_in_use = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device_in_use)

cuda


In [2]:
# Load the data
ch = load_breast_cancer()
df = pd.DataFrame(data=ch.data, columns=ch.feature_names)
# Assuming `ch.target` is the target variable
df['Target'] = ch.target

# Splitting the dataset
df_train, df_temp = train_test_split(df, train_size=0.70, random_state=42)
df_val, df_test = train_test_split(df_temp, train_size=0.5, random_state=42)

# Separate the target variable
y_train = df_train['Target'].values
y_val = df_val['Target'].values
y_test = df_test['Target'].values

# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler to the training data (excluding the target variable) and transform it
scaled_train_features = scaler.fit_transform(df_train.drop(columns=['Target']))

# Transform the validation and test data (excluding the target variable)
scaled_val_features = scaler.transform(df_val.drop(columns=['Target']))
scaled_test_features = scaler.transform(df_test.drop(columns=['Target']))

# Recombine scaled features with target variable
df_scaled_train = pd.DataFrame(scaled_train_features, columns=df_train.columns[:-1])  # Excluding the target variable column
df_scaled_train['Target'] = y_train

df_scaled_val = pd.DataFrame(scaled_val_features, columns=df_val.columns[:-1])
df_scaled_val['Target'] = y_val

df_scaled_test = pd.DataFrame(scaled_test_features, columns=df_test.columns[:-1])
df_scaled_test['Target'] = y_test

class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.features = dataframe.drop('Target', axis=1).values
        self.labels = dataframe['Target'].values

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return torch.tensor(self.features[idx], dtype=torch.float), torch.tensor(self.labels[idx], dtype=torch.long)

train_dataset = CustomDataset(df_scaled_train)
val_dataset = CustomDataset(df_scaled_val)
test_dataset = CustomDataset(df_scaled_test)

train_loader = DataLoader(train_dataset, batch_size=len(train_dataset), shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)

In [3]:
# Define a function to move an entire dataset to the device in advance
def preload_dataset_to_device(loader, batch_size, device):
    preloaded_data = [(inputs.to(device), targets.to(device)) for inputs, targets in loader]
    return DataLoader(preloaded_data, batch_size=batch_size)

# Preload datasets to device (if they fit into your device memory)
train_loader = preload_dataset_to_device(train_loader, len(train_dataset), device_in_use)
val_loader = preload_dataset_to_device(val_loader, len(val_dataset), device_in_use)

In [5]:
input_size = scaled_train_features.shape[1]
output_size = 2
test_loss_dic, train_loss_dic = weight_avg_8_classification(200, train_loader, val_loader, input_size, output_size, device_in_use, model = 'NN') 

test_loss_dic

{'m1_0': 0.6478309035301208,
 'm1_1': 0.6183509230613708,
 'm1_2': 0.5915619134902954,
 'm1_3': 0.5671729445457458,
 'm1_4': 0.5449219942092896,
 'm1_5': 0.5245744585990906,
 'm1_6': 0.505922794342041,
 'm1_7': 0.48878300189971924,
 'm1_8': 0.47299352288246155,
 'm1_9': 0.45841184258461,
 'm1_10': 0.4449131190776825,
 'm1_11': 0.4323871433734894,
 'm1_12': 0.4207371473312378,
 'm2_0': 0.6653599739074707,
 'm2_1': 0.6349195837974548,
 'm2_2': 0.6071872711181641,
 'm2_3': 0.5818769931793213,
 'm2_4': 0.5587323307991028,
 'm2_5': 0.5375235676765442,
 'm2_6': 0.5180463790893555,
 'm2_7': 0.5001193284988403,
 'm2_8': 0.4835819602012634,
 'm2_9': 0.4682918190956116,
 'm2_10': 0.45412346720695496,
 'm2_11': 0.4409656822681427,
 'm2_12': 0.42871996760368347,
 'm3_0': 0.582017719745636,
 'm3_1': 0.5560950040817261,
 'm3_2': 0.5329479575157166,
 'm3_3': 0.5121691823005676,
 'm3_4': 0.493415504693985,
 'm3_5': 0.47640183568000793,
 'm3_6': 0.4608910083770752,
 'm3_7': 0.44668641686439514,
 'm3_8'

In [6]:
def keywithminval(d):
     """ a) create a list of the dict's keys and values; 
         b) return the key with the max value"""  
     v = list(d.values())
     k = list(d.keys())
     return k[v.index(min(v))], min(v)

keywithminval(test_loss_dic)

('m3_12', 0.3903517425060272)

In [7]:
input_size = scaled_train_features.shape[1]
output_size = 2

train_loss_traditional = []
test_loss_traditional = []

train_loss_2 = []
test_loss_2 = []

train_loss_8 = []
test_loss_8 = []

#105 is the lcm(3,7,15)

from tqdm import tqdm
for i in tqdm(range(500)):

    test_loss_dic, train_loss_dic = regular_classification(3*15*5, train_loader, val_loader, input_size, output_size, device_in_use, model = 'NN') 
    _, testval = keywithminval(test_loss_dic)
    _, trainval = keywithminval(train_loss_dic)

    train_loss_traditional.append(trainval) 
    test_loss_traditional.append(testval)

    test_loss_dic, train_loss_dic = weight_avg_2_classification(3*15*5, train_loader, val_loader, input_size, output_size, device_in_use, model = 'NN') 

    _, testval = keywithminval(test_loss_dic)
    _, trainval = keywithminval(train_loss_dic)

    train_loss_2.append(trainval) 
    test_loss_2.append(testval)

    test_loss_dic, train_loss_dic = weight_avg_8_classification(3*15*5, train_loader, val_loader, input_size, output_size, device_in_use, model = 'NN') 

    _, testval = keywithminval(test_loss_dic)
    _, trainval = keywithminval(train_loss_dic)

    train_loss_8.append(trainval) 
    test_loss_8.append(testval)


print("Traditional:",np.mean(test_loss_traditional),np.std(test_loss_traditional))
print("Weight Avg 2 Base Models:",np.mean(test_loss_2),np.std(test_loss_2))
print("Weight Avg 8 Base Models:",np.mean(test_loss_8),np.std(test_loss_8))

100%|██████████| 500/500 [19:59<00:00,  2.40s/it]

Traditional: 0.1265121241658926 0.002562410141505997
Weight Avg 2 Base Models: 0.19358987566828728 0.0035348763757059426
Weight Avg 8 Base Models: 0.36943213653564455 0.010771749405481638


In [9]:
input_size = scaled_train_features.shape[1]
output_size = 2

train_loss_traditional = []
test_loss_traditional = []

train_loss_2 = []
test_loss_2 = []

train_loss_8 = []
test_loss_8 = []

#105 is the lcm(3,7,15)

from tqdm import tqdm
for i in tqdm(range(500)):

    test_loss_dic, train_loss_dic = regular_classification(3*15*5, train_loader, val_loader, input_size, output_size, device_in_use, model = 'Linear') 
    _, testval = keywithminval(test_loss_dic)
    _, trainval = keywithminval(train_loss_dic)

    train_loss_traditional.append(trainval) 
    test_loss_traditional.append(testval)

    test_loss_dic, train_loss_dic = weight_avg_2_classification(3*15*5, train_loader, val_loader, input_size, output_size, device_in_use, model = 'Linear') 

    _, testval = keywithminval(test_loss_dic)
    _, trainval = keywithminval(train_loss_dic)

    train_loss_2.append(trainval) 
    test_loss_2.append(testval)

    test_loss_dic, train_loss_dic = weight_avg_8_classification(3*15*5, train_loader, val_loader, input_size, output_size, device_in_use, model = 'Linear') 

    _, testval = keywithminval(test_loss_dic)
    _, trainval = keywithminval(train_loss_dic)

    train_loss_8.append(trainval) 
    test_loss_8.append(testval)


print("Traditional:",np.mean(test_loss_traditional),np.std(test_loss_traditional))
print("Weight Avg 2 Base Models:",np.mean(test_loss_2),np.std(test_loss_2))
print("Weight Avg 8 Base Models:",np.mean(test_loss_8),np.std(test_loss_8))

100%|██████████| 500/500 [18:04<00:00,  2.17s/it]

Traditional: 0.12413684768974781 0.004761746924787912
Weight Avg 2 Base Models: 0.14461213859915734 0.004461780008218083
Weight Avg 8 Base Models: 0.21363147780299185 0.005292064879879165
